In [1]:
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

pd.set_option('max_colwidth', 200)
pd.options.display.max_rows = 200

In [2]:
from const import *
import train_reorder_size_model
import utils, predict

/home/damonhao/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df_analysis = predict.read_max_f1_analysis('../prediction/analysis_08-04-20-56-54_validate_diff_stack.csv')

In [4]:
df_analysis_real = predict.read_max_f1_analysis('../prediction/analysis_08-04-20-54-13_validate_diff_stack.csv')

In [16]:
df_predict_reorder_size = pd.read_csv('../prediction/predict_validate_reorder_size.csv')

In [8]:
df_comp_real = df_analysis.merge(df_analysis_real, how='left', on=[UID], suffixes=['_var', '_real'])

In [45]:
df_comp_real[df_comp_real.scores_real < df_comp_real.scores_var].scores_var.sum() / df_comp_real[df_comp_real.scores_real < df_comp_real.scores_var].shape[0]

0.470382645137248

In [9]:
df_real_greate = df_comp_real[df_comp_real.scores_real > df_comp_real.scores_var]

In [46]:
df_real_less = df_comp_real[df_comp_real.scores_real < df_comp_real.scores_var]

In [10]:
df_real_greate.shape

(12207, 9)

In [19]:
real_bold_index = df_real_greate.apply(lambda row: len(row['products_real']) > len(row['products_var']) , axis=1)

In [31]:
df_real_greate_bold = df_real_greate[real_bold_index]

In [32]:
(df_real_greate_bold.scores_real - df_real_greate_bold.scores_var).sum() / float(df_real_greate_bold.shape[0])

0.13587809158562306

In [33]:
df_predict_reorder_size[df_predict_reorder_size.user_id.isin(df_real_greate_bold.user_id)].reorder_size.describe()

count    6030.000000
mean       11.807629
std         7.066314
min         2.000000
25%         7.000000
50%        10.000000
75%        15.000000
max        71.000000
Name: reorder_size, dtype: float64

In [40]:
df_predict_reorder_size.columns

Index([u'user_id', u'reorder_size', u'pred_reorder_size_float',
       u'pred_reorder_size'],
      dtype='object')

In [41]:
df_predict_reorder_size[df_predict_reorder_size.user_id.isin(df_real_greate_bold.user_id)].pred_reorder_size.describe()

count    6030.000000
mean        7.169818
std         4.761419
min         1.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        42.000000
Name: pred_reorder_size, dtype: float64

In [36]:
real_care_index = df_real_greate.apply(lambda row: len(row['products_real']) < len(row['products_var']) and row['true_products_var'][0] is not None , axis=1)

In [37]:
df_real_greate_care = df_real_greate[real_care_index]

In [38]:
(df_real_greate_care.scores_real - df_real_greate_care.scores_var).sum() / float(df_real_greate_care.shape[0])

0.14449750086571694

In [39]:
df_predict_reorder_size[df_predict_reorder_size.user_id.isin(df_real_greate_care.user_id)].reorder_size.describe()

count    4446.000000
mean        4.721098
std         3.692334
min         1.000000
25%         2.000000
50%         4.000000
75%         6.000000
max        39.000000
Name: reorder_size, dtype: float64

In [42]:
df_predict_reorder_size[df_predict_reorder_size.user_id.isin(df_real_greate_care.user_id)].pred_reorder_size.describe()

count    4446.000000
mean        7.120558
std         4.438845
min         1.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        45.000000
Name: pred_reorder_size, dtype: float64

In [50]:
none_index = df_comp_real.apply(lambda row: True if row['true_products_var'][0] is None else False, axis=1)

In [53]:
df_comp_real_none = df_comp_real[none_index]

In [91]:
df_comp_real_none[df_comp_real_none.scores_var == 0].tail(10)

,user_id,true_products_var,products_var,best_k_proba_var,scores_var,true_products_real,products_real,best_k_proba_real,scores_real
25530,189065,[None],"[30406, 27104]",[ 0.50445 0.50211],0.0,[None],[None],[ nan],1.0
25533,167934,[None],"[47766, 28515]",[ 0.50374 0.48597],0.0,[None],[None],[ nan],1.0
25541,193137,[None],"[10054, 21351]",[ 0.5021 0.48128],0.0,[None],[None],[ nan],1.0
25543,176196,[None],"[1117, 18465]",[ 0.50182 0.486 ],0.0,[None],[None],[ nan],1.0
25549,192204,[None],"[34126, 16083, 31717, 32299, 651]",[ 0.50128 0.48557 0.46073 0.42392 0.39509],0.0,[None],[None],[ nan],1.0
25554,185110,[None],"[43892, 12341]",[ 0.50015 0.48564],0.0,[None],[None],[ nan],1.0
25601,175116,[None],"[6299, 47626]",[ 0.49105 0.48764],0.0,[None],[None],[ nan],1.0
25606,194170,[None],"[25931, 18465, 34943]",[ 0.49017 0.42695 0.36559],0.0,[None],[None],[ nan],1.0
25646,190261,[None],"[13974, 24852, 43205]",[ 0.48458 0.45699 0.27818],0.0,[None],[None],[ nan],1.0
26042,179181,[None],"[260, 21019, 47766, 44632, 46906]",[ 0.40907 0.40004 0.3717 0.36942 0.36065],0.0,[None],[None],[ nan],1.0


In [67]:
df_comp_real_not_none = df_comp_real[~none_index]

In [72]:
df_comp_real_not_none.shape

(24617, 9)

In [73]:
df_real_not_none_greate = df_comp_real_not_none[df_comp_real_not_none.scores_real > df_comp_real_not_none.scores_var]

In [84]:
def real_bold_func(row):
    return len(row['products_real']) > len(row['products_var'])

def real_care_func(row):
    return len(row['products_real']) < len(row['products_var'])

In [75]:
not_none_real_bold_index = df_real_not_none_greate.apply(
    lambda row: len(row['products_real']) > len(row['products_var']), axis=1)
df_real_not_none_greate_bold = df_real_not_none_greate[not_none_real_bold_index]

In [78]:
df_real_not_none_greate.shape

(10504, 9)

In [76]:
df_real_not_none_greate_bold.shape

(6030, 9)

In [81]:
df_real_not_none_less = df_comp_real_not_none[df_comp_real_not_none.scores_real < df_comp_real_not_none.scores_var]

In [85]:
real_not_none_less_care_index = df_real_not_none_less.apply(real_care_func, axis=1)

In [87]:
df_real_not_none_less_care = df_real_not_none_less[real_not_none_less_care_index]

In [89]:
df_real_not_none_less_bold = df_real_not_none_less[df_real_not_none_less.apply(real_bold_func, axis=1)]

In [90]:
df_real_not_none_less_bold.head()

,user_id,true_products_var,products_var,best_k_proba_var,scores_var,true_products_real,products_real,best_k_proba_real,scores_real
5,169442,"[42342, 38772, 28928, 47792, 5876, 8174, 13176, 40706, 30391, 43789, 35946, 19057]","[13176, 42342, 28928, 38772, 40706, 5876, 47792, 30391, 31040, 19057, 43789]",[ 0.92799 0.91975 0.90225 0.90096 0.88249 0.87898 0.87424 0.87173\n 0.84921 0.80945 0.79494],0.869565,"[42342, 38772, 28928, 47792, 5876, 8174, 13176, 40706, 30391, 43789, 35946, 19057]","[13176, 42342, 28928, 38772, 40706, 5876, 47792, 30391, 31040, 19057, 43789, 21137]",[ 0.92799 0.91975 0.90225 0.90096 0.88249 0.87898 0.87424 0.87173\n 0.84921 0.80945 0.79494 0.79368],0.833333
9,204317,"[33198, 12606, 25890]","[33198, 12606]",[ 0.9274 0.90733],0.800000,"[33198, 12606, 25890]","[33198, 12606, 1194]",[ 0.9274 0.90733 0.79162],0.666667
13,165978,"[33827, 47626, 24852, 3023, 39408, 23322, 21137, 28204, 9362, 38731]","[24852, 47626, 33827, 23322, 39408, 21137, 3023, 25706, 28204]",[ 0.9273 0.92475 0.92439 0.90602 0.89587 0.88451 0.87292 0.85317\n 0.84342],0.842105,"[33827, 47626, 24852, 3023, 39408, 23322, 21137, 28204, 9362, 38731]","[24852, 47626, 33827, 23322, 39408, 21137, 3023, 25706, 28204, 16489]",[ 0.9273 0.92475 0.92439 0.90602 0.89587 0.88451 0.87292 0.85317\n 0.84342 0.71182],0.800000
14,174647,"[5876, 39928, 13176, 14966, 39877, 37646, 43122, 22825, 4920]","[13176, 39877, 37646, 31915, 43122, 8174, 22825, 14966]",[ 0.92722 0.91539 0.90459 0.9018 0.89793 0.8972 0.89583 0.87247],0.705882,"[5876, 39928, 13176, 14966, 39877, 37646, 43122, 22825, 4920]","[13176, 39877, 37646, 31915, 43122, 8174, 22825, 14966, 21288]",[ 0.92722 0.91539 0.90459 0.9018 0.89793 0.8972 0.89583 0.87247\n 0.85043],0.666667
29,165956,"[13176, 35649, 14947, 47209, 39877, 18105, 9839, 31915, 16797, 21903, 10246, 36956, 15290, 18465, 41658, 35958, 27966, 38456, 5876, 10017, 39475, 27999, 26940]","[13176, 35649, 39877, 47209, 14947, 31915, 21903, 9839, 18105, 27966, 38456, 39475, 27999, 16797, 5876, 10017, 17459, 35958, 18465, 27104]",[ 0.92675 0.92427 0.92423 0.92408 0.92405 0.92366 0.92302 0.91951\n 0.91148 0.90658 0.89903 0.89329 0.88756 0.8839 0.87796 0.87617\n 0.84065 0.83333 0.83194 0.76125],0.837209,"[13176, 35649, 14947, 47209, 39877, 18105, 9839, 31915, 16797, 21903, 10246, 36956, 15290, 18465, 41658, 35958, 27966, 38456, 5876, 10017, 39475, 27999, 26940]","[13176, 35649, 39877, 47209, 14947, 31915, 21903, 9839, 18105, 27966, 38456, 39475, 27999, 16797, 5876, 10017, 17459, 35958, 18465, 27104, 25197, 44306, 1819]",[ 0.92675 0.92427 0.92423 0.92408 0.92405 0.92366 0.92302 0.91951\n 0.91148 0.90658 0.89903 0.89329 0.88756 0.8839 0.87796 0.87617\n 0.84065 0.83333 0.83194 0.76125 0.72711 0...,0.782609


In [79]:
df_real_not_none_greate[~not_none_real_bold_index].iloc[0: 50]

,user_id,true_products_var,products_var,best_k_proba_var,scores_var,true_products_real,products_real,best_k_proba_real,scores_real
10,203305,"[43154, 19660, 1938]","[43154, 28204, 19660, 45007, 38383]",[ 0.9274 0.91534 0.8919 0.8046 0.75067],0.500000,"[43154, 19660, 1938]","[43154, 28204, 19660]",[ 0.9274 0.91534 0.8919 ],0.666667
23,194570,[32403],"[32403, 8048]",[ 0.92684 0.51853],0.666667,[32403],[32403],[ 0.92684],1.000000
28,186005,"[19660, 37646, 14084, 46969, 30391, 8174, 45364, 39911, 39886, 11520, 49175, 17948, 27156]","[13176, 37646, 19660, 14084, 30391, 46969, 47766, 45007, 39928, 42768, 11520, 8174, 45364, 49175, 2825, 17948, 42265, 41218, 29439, 10644]",[ 0.92675 0.92478 0.92465 0.92388 0.92278 0.9208 0.9169 0.91505\n 0.91068 0.90799 0.90382 0.9033 0.90123 0.88848 0.83271 0.82894\n 0.80721 0.80457 0.79744 0.76781],0.606061,"[19660, 37646, 14084, 46969, 30391, 8174, 45364, 39911, 39886, 11520, 49175, 17948, 27156]","[13176, 37646, 19660, 14084, 30391, 46969, 47766, 45007, 39928, 42768, 11520, 8174, 45364]",[ 0.92675 0.92478 0.92465 0.92388 0.92278 0.9208 0.9169 0.91505\n 0.91068 0.90799 0.90382 0.9033 0.90123],0.615385
36,203356,"[24852, 9292, 47766, 19057, 6184]","[24852, 47766, 9292, 19057, 6184, 6155]",[ 0.92649 0.91372 0.90126 0.89587 0.89237 0.77606],0.909091,"[24852, 9292, 47766, 19057, 6184]","[24852, 47766, 9292, 19057, 6184]",[ 0.92649 0.91372 0.90126 0.89587 0.89237],1.000000
47,198897,"[43154, 21572, 196, 13042, 130, 31759, 10310]","[43154, 38928, 48034, 45051, 196, 25146, 116, 10258]",[ 0.92624 0.91008 0.90257 0.8533 0.82174 0.81188 0.7554 0.72138],0.266667,"[43154, 21572, 196, 13042, 130, 31759, 10310]","[43154, 38928, 48034, 45051, 196, 25146, 116]",[ 0.92624 0.91008 0.90257 0.8533 0.82174 0.81188 0.7554 ],0.285714
51,202470,"[22537, 13176, 17948, 39877, 34126, 39984, 3952, 34134, 14211, 46979, 43961]","[13176, 21405, 39877, 22537, 34126, 43961, 17948, 46979, 21903, 3952, 39984, 22825, 48679, 34134, 35221]",[ 0.92621 0.9164 0.91489 0.91224 0.90653 0.88961 0.8884 0.8878\n 0.88479 0.88306 0.85286 0.8352 0.82262 0.81275 0.79247],0.769231,"[22537, 13176, 17948, 39877, 34126, 39984, 3952, 34134, 14211, 46979, 43961]","[13176, 21405, 39877, 22537, 34126, 43961, 17948, 46979, 21903, 3952, 39984]",[ 0.92621 0.9164 0.91489 0.91224 0.90653 0.88961 0.8884 0.8878\n 0.88479 0.88306 0.85286],0.818182
69,191767,"[196, 43154]","[43154, 196, 12341]",[ 0.92589 0.92455 0.77076],0.800000,"[196, 43154]","[43154, 196]",[ 0.92589 0.92455],1.000000
71,185031,"[24852, 28204, 26604, 10603, 28985, 4920, 16797, 21903, 47144, 23375, 46802]","[24852, 10603, 28985, 47144, 4920, 26604, 28204, 21903, 16797, 23375, 46802, 41787]",[ 0.92583 0.92241 0.92118 0.92111 0.91923 0.91856 0.91568 0.91143\n 0.91061 0.88998 0.8826 0.71879],0.956522,"[24852, 28204, 26604, 10603, 28985, 4920, 16797, 21903, 47144, 23375, 46802]","[24852, 10603, 28985, 47144, 4920, 26604, 28204, 21903, 16797, 23375, 46802]",[ 0.92583 0.92241 0.92118 0.92111 0.91923 0.91856 0.91568 0.91143\n 0.91061 0.88998 0.8826 ],1.000000
75,180582,"[24852, 28204, 11123, 35042, 47766, 13396, 30849, 35163]","[24852, 11123, 47766, 17615, 30849, 35042, 8424, 12496, 36011, 13396, 38995, 47144, 5077]",[ 0.92579 0.92444 0.91519 0.90747 0.8932 0.89158 0.88276 0.88036\n 0.87734 0.85819 0.84317 0.82222 0.81428],0.571429,"[24852, 28204, 11123, 35042, 47766, 13396, 30849, 35163]","[24852, 11123, 47766, 17615, 30849, 35042, 8424, 12496]",[ 0.92579 0.92444 0.91519 0.90747 0.8932 0.89158 0.88276 0.88036],0.625000
80,179651,"[41290, 46886, 28745]","[41290, 28745, 46699, 40198]",[ 0.92569 0.8499 0.68709 0.65251],0.571429,"[41290, 46886, 28745]","[41290, 28745, 46699]",[ 0.92569 0.8499 0.68709],0.666667


# 验证 None 作用

In [58]:
df_analysis_copy = df_analysis.copy()

In [65]:
df_analysis_copy.ix[df_comp_real_none.index, 'scores'] = 1

In [66]:
df_analysis_copy.scores.mean()

0.4290489921055521

In [92]:
df_train = pd.read_csv('../reorder_size_input/train_all_reorder.csv')

In [93]:
df_train[df_train.reorder_size == 0].shape[0] / float(df_train.shape[0])

0.06515040212175273

In [94]:
df_validate = pd.read_csv('../reorder_size_input/validate_all_reorder.csv')

In [95]:
df_validate[df_validate.reorder_size == 0].shape[0] / float(df_validate.shape[0])

0.0671845395983327

In [96]:
(df_validate[df_validate.reorder_size == 0].shape[0] + df_train[df_train.reorder_size == 0].shape[0]) / float(df_train.shape[0]+df_validate.shape[0])

0.06555952716658156

In [13]:
df_wd_none = predict.read_max_f1_analysis('../prediction/analysis_08-08-22-21-50_validate_diff_stack.csv')

In [12]:
df_w_none = predict.read_max_f1_analysis('../prediction/analysis_08-08-22-23-14_validate_diff_stack.csv')

In [14]:
df_comp_none = df_wd_none.merge(df_w_none, how='left', on=[UID], suffixes=['_wdNone', '_wNone'])

In [15]:
none_index = df_comp_none.apply(lambda row: True if row['true_products_wdNone'][0] is None else False, axis=1)

In [16]:
df_none = df_comp_none[none_index]

In [27]:
a = df_none[df_none.scores_wdNone < df_none.scores_wNone]
print a.shape
print (a.scores_wNone - a.scores_wdNone).sum()

(855, 9)
421.603246753


In [29]:
b = df_comp_none[df_comp_none.scores_wNone < df_comp_none.scores_wdNone]
print b.shape
print (b.scores_wdNone - b.scores_wNone).sum()

(2629, 9)
285.518498457
